<a href="https://colab.research.google.com/github/spoon-while-spoon/bike-traffic-mlpred/blob/main/data_preperation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning anhand von Radverkehrsdaten

In [ ]:
# Importieren der benötigten Bibliotheken
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline
sns.set(color_codes=True) # Setzt Standardfarbcodes für Seaborn-Plots

# Konvertiert einen Integer-Zeitstempel in ein Datums-/Zeitformat
def parse_date(date_int):
    date_str = str(date_int) # Konvertiert den Integer in einen String
    return pd.to_datetime(date_str, format='%Y%m%d%H')

# Einlesen der CSV-Dateien sowie erste Anpassungen
wind_df = pd.read_csv("wind.csv", sep=";", index_col=1, parse_dates=True, date_parser=parse_date).drop(['eor', 'STATIONS_ID', 'QUALITAETS_NIVEAU', 'STRUKTUR_VERSION', 'WINDRICHTUNG'], axis=1)
sun_df = pd.read_csv("sun.csv", sep=";", index_col=1, parse_dates=True, date_parser=parse_date).drop(['STATIONS_ID', 'QUALITAETS_NIVEAU', 'STRUKTUR_VERSION', 'eor'], axis=1)
precipation_df = pd.read_csv("precipation.csv", sep=";", index_col=1, parse_dates=True, date_parser=parse_date).drop(['STATIONS_ID', 'QUALITAETS_NIVEAU', 'STRUKTUR_VERSION', 'eor'], axis=1)
airpressure_df = pd.read_csv("airpressure.csv", sep=";", index_col=1, parse_dates=True, date_parser=parse_date).drop(['STATIONS_ID', 'QUALITAETS_NIVEAU', 'STRUKTUR_VERSION', 'LUFTDRUCK_STATIONSHOEHE', 'eor'], axis=1)

air_temp_df = pd.read_csv("air_temp.csv", sep=";", index_col=1, parse_dates=True, date_parser=parse_date).drop(['STATIONS_ID', 'QUALITAETS_NIVEAU', 'STRUKTUR_VERSION', 'LUFTTEMPERATUR_FALSCH', 'STRAHLUNGSTEMPERATUR', 'eor'], axis=1)
air_temp_df['LUFTTEMPERATUR_RICHTIG'] = pd.to_numeric(air_temp_df['LUFTTEMPERATUR_RICHTIG'], errors='coerce')
air_temp_df['REL_FEUCHTE'] = pd.to_numeric(air_temp_df['REL_FEUCHTE'], errors='coerce')
air_temp_df = air_temp_df[(air_temp_df['LUFTTEMPERATUR_RICHTIG'] >= -25) & (air_temp_df['LUFTTEMPERATUR_RICHTIG'] <= 45) & (air_temp_df['REL_FEUCHTE'] >= 0) & (air_temp_df['REL_FEUCHTE'] <= 99)]

radverkehr_df = pd.read_csv("radverkehr_final.csv", sep=";", index_col=1, parse_dates=True, date_parser=parse_date).drop(['datum_rad'], axis=1).assign(RAD_gesamt=lambda df: df['NORD'] + df['SUED'])



### Kombinieren & Bereinigen

Hier werden unsere erstellten Dataframes zu einem **kombiniert** `(pd.concat)` und anschließend  **bereinigt** von unvollständigen Datensätzen.

Datenreihen zu denen nicht alle Daten vorliegen werden komplett entfernt durch `dropna()`.



In [ ]:
df_cleaned = pd.concat([wind_df, sun_df, precipation_df, airpressure_df, air_temp_df, radverkehr_df], axis=1).dropna()

### Ergebnis
Mit `print(df_cleaned.head())` lassen sich die ersten Datenreihen überprüfgen.
Die Anzahl der Übrig gebliebenen Datensätze kann mit `num_rows = len(df_cleaned)` gezählt und dann ausgeben werden.  

In [ ]:
print(df_cleaned.head())
print("\n--------------------------------------------------")
print("Gesamtzahl der bereinigten Datensätze:", num_rows)
print("--------------------------------------------------\n")

                     WINDGESCHWINDIGKEIT  SONNENSCHEINDAUER  \
MESS_DATUM                                                    
2015-08-20 05:00:00                860.0                0.0   
2015-08-20 10:00:00                480.0               45.0   
2015-08-20 19:00:00                420.0                0.0   
2015-08-21 00:00:00                920.0                0.0   
2015-08-21 03:00:00                800.0                0.0   

                    NIEDERSCHLAGSHOEHE  LUFTDRUCK_NN  LUFTTEMPERATUR_RICHTIG  \
MESS_DATUM                                                                     
2015-08-20 05:00:00                  0       10237.0                    15.0   
2015-08-20 10:00:00                  0       10244.0                    24.0   
2015-08-20 19:00:00                  0       10246.0                    23.0   
2015-08-21 00:00:00                  0       10255.0                    19.0   
2015-08-21 03:00:00                  0       10255.0                    17.0  

In [ ]:
df_cleaned.to_csv('/content/drive/My Drive/df_cleaned.csv', index=True, sep=';')